In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import joblib
from tqdm import tqdm
import pandas as pd
import librosa.display
import matplotlib.pyplot as plt
from pathlib import Path
import soundfile as sf  # Para guardar en formato WAV
from sklearn.metrics import recall_score, f1_score


In [23]:
def extraer_caracteristicas_1(ruta_completa,clase):
    y, sr = librosa.load(ruta_completa)

    # Extracción de características
    mean = np.mean(y)
    std = np.std(y)


    #Calcular los Mel-Frequency Cepstral Coefficients (MFCC)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    #Obtener el promedio de los coeficientes MFCC
    mean_mfccs = np.mean(mfccs, axis=1)
     #primera y segunda derivada
    #delta_mfcc = librosa.feature.delta(mfccs)
    #delta_mfcc_2 = librosa.feature.delta(mfccs, order=2)
     #Características en el dominio de la frecuencia
    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)[0])
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr)[0])
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr)[0])

    #Características de energía
    zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=y)[0])

    clase=clase



    # Crear una fila con todos los datos
    caracteristicas = [

        *mean_mfccs,#Desempaqueta
        spectral_centroid,
        spectral_bandwidth,
        spectral_rolloff,
        zero_crossing_rate,



        clase
    ]
    return caracteristicas


In [24]:
data = []

In [25]:
# Lista para almacenar las filas del dataset hombre y mujeres enfermos
#data = []
directorio_audio="/content/drive/MyDrive/PROYECTO_FINAL/Frases_enfermos_HM_duplicado/export"
clase="sick"
# Cargar el archivo CSV con los nombres de los archivos y las clases
#metadata = pd.read_csv(ruta_metadata)

# Iterar sobre los archivos de audio en el directorio
for archivo_audio in tqdm(os.listdir(directorio_audio)):
        if archivo_audio.endswith('.wav'):  # Asegurarse de que estás trabajando con archivos de audio
            ruta_completa = os.path.join(directorio_audio, archivo_audio)
            # Extraer características 1
            fila = extraer_caracteristicas_1(ruta_completa,clase)

            # Agregar la fila a la lista de datos
            data.append(fila)

       # nombres_columnas = [
       # 'Ruta',
       # 'Media señal',
       # 'Desviación estándar de la señal',
       # 'MFCC_1', 'MFCC_2', 'MFCC_3', 'MFCC_4', 'MFCC_5', 'MFCC_6', 'MFCC_7', 'MFCC_8', 'MFCC_9', 'MFCC_10', 'MFCC_11', 'MFCC_12', 'MFCC_13',
       # 'Amplitud máxima',
       # 'Amplitud mínima',
       # 'Promedio del centroides espectrales',
       # 'Promedio del ancho de banda espectral',
       # 'Promedio del rolloff espectral',
       # 'Promedio de la tasa de cruce por cero',
       # 'RMS (Root Mean Square)',
       # 'Clase'
   ## ]

#df = pd.DataFrame(data, columns=nombres_columnas)

    # Guardar el DataFrame como un archivo CSV
#df.to_csv('dataset_test_1.csv', index=False)


100%|██████████| 1724/1724 [01:46<00:00, 16.22it/s]


In [26]:
# Lista para almacenar las filas del dataset hombre y mujeres sanas
#data = []
directorio_audio="/content/drive/MyDrive/PROYECTO_FINAL/frases sanos + whitenoise sanos"
clase="sano"
# Cargar el archivo CSV con los nombres de los archivos y las clases
#metadata = pd.read_csv(ruta_metadata)

# Iterar sobre los archivos de audio en el directorio
for archivo_audio in tqdm(os.listdir(directorio_audio)):
        if archivo_audio.endswith('.wav'):  # Asegurarse de que estás trabajando con archivos de audio
            ruta_completa = os.path.join(directorio_audio, archivo_audio)
            # Extraer características 1
            fila = extraer_caracteristicas_1(ruta_completa,clase)

            # Agregar la fila a la lista de datos
            data.append(fila)

100%|██████████| 1656/1656 [01:26<00:00, 19.17it/s]


In [27]:
nombres_columnas = [
        'Ruta',
        'Media señal',
        'Dstandar',#Desviación estándar de la señal
        'MFCC_1', 'MFCC_2', 'MFCC_3', 'MFCC_4', 'MFCC_5', 'MFCC_6', 'MFCC_7', 'MFCC_8', 'MFCC_9', 'MFCC_10', 'MFCC_11', 'MFCC_12', 'MFCC_13',
        'Amáxima',#Amplitud maxima
        'Amín',#Amplitud mínima
        'AvgCS', # Promedio del centroides espectrales
        'AvgBws',#Promedio del ancho de banda espectral
        'Avgrolloff',#Promedio del rolloff espectral
        'AvgCrossZ',#Promedio de la tasa de cruce por cero
        'RMS ',#RMS (Root Mean Square)
        'Clase'
    ]

df = pd.DataFrame(data)

    # Guardar el DataFrame como un archivo CSV
#df.to_csv('dataset_test_1.csv', index=False)


In [28]:
print(df)

              0           1          2          3          4          5   \
0    -265.925293  100.256241  19.701265  36.082523   5.650431  -5.365362   
1    -282.439087  109.824287  17.232365  32.695984   6.277915 -19.289959   
2    -256.304932  108.921295  36.863407  21.057262  -1.506584  -3.437346   
3    -254.603073  117.361710  40.523499  32.599110   3.194947 -11.256540   
4    -261.194397   68.744141  30.958025  24.921591  15.521741 -16.009760   
...          ...         ...        ...        ...        ...        ...   
3375 -284.316193   79.033096  24.230682  33.330681  16.543144  -0.144021   
3376 -206.528320  106.611496  21.443638  40.453037   8.335456  -8.586728   
3377 -265.965698   93.998299  30.081154  32.448971  12.935879   1.060116   
3378 -284.963531   88.529770  24.216827  30.196507  10.512567  -2.964409   
3379 -297.610260  110.968552  37.173805  42.027958  11.998269  -0.544153   

            6          7          8          9          10        11  \
0    -5.256649 

In [29]:
#directorio_audio="/content/drive/MyDrive/PROYECTO_FINAL/PATH/Hombre_frases_path"
###clase="sick"
#recorrer_archivos_audio(directorio_audio,clase)


In [30]:
#print(df.describe())

In [31]:
df[17] = df[17].map({'sick': 1, 'sano': 0})

In [32]:

import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler



# Estandarizar el DataFrame (opcional pero recomendado antes de PCA)
scaler = StandardScaler()
df_estandarizado = scaler.fit_transform(df)

#Instanciar PCA y especificar el número de componentes deseados
n_componentes = 2  # Número de componentes principales
pca = PCA(n_components=n_componentes)

# Aplicar PCA al DataFrame estandarizado
df_pca = pd.DataFrame(pca.fit_transform(df_estandarizado), columns=[f'PC{i}' for i in range(1, n_componentes + 1)])

#Mostrar el DataFrame después de PCA
print("DataFrame Original:")
print(df)
#print("\nDataFrame después de PCA:")
print(df_pca)
#print(df_estandarizado)

DataFrame Original:
              0           1          2          3          4          5   \
0    -265.925293  100.256241  19.701265  36.082523   5.650431  -5.365362   
1    -282.439087  109.824287  17.232365  32.695984   6.277915 -19.289959   
2    -256.304932  108.921295  36.863407  21.057262  -1.506584  -3.437346   
3    -254.603073  117.361710  40.523499  32.599110   3.194947 -11.256540   
4    -261.194397   68.744141  30.958025  24.921591  15.521741 -16.009760   
...          ...         ...        ...        ...        ...        ...   
3375 -284.316193   79.033096  24.230682  33.330681  16.543144  -0.144021   
3376 -206.528320  106.611496  21.443638  40.453037   8.335456  -8.586728   
3377 -265.965698   93.998299  30.081154  32.448971  12.935879   1.060116   
3378 -284.963531   88.529770  24.216827  30.196507  10.512567  -2.964409   
3379 -297.610260  110.968552  37.173805  42.027958  11.998269  -0.544153   

            6          7          8          9          10        1

In [33]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Instanciar el MinMaxScaler
scaler = MinMaxScaler()

# Ajustar y transformar el DataFrame
df_normalizado = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

# Mostrar el DataFrame normalizado

print("\nDataFrame Normalizado:")
print(df_normalizado)


DataFrame Normalizado:
            0         1         2         3         4         5         6   \
0     0.624873  0.526329  0.447838  0.622752  0.385414  0.455559  0.456758   
1     0.579063  0.598038  0.413062  0.562488  0.394287  0.198875  0.510111   
2     0.651560  0.591270  0.689575  0.355375  0.284202  0.491099  0.374767   
3     0.656281  0.654529  0.741129  0.560764  0.350689  0.346962  0.545127   
4     0.637996  0.290156  0.606395  0.424142  0.525009  0.259342  0.666541   
...        ...       ...       ...       ...       ...       ...       ...   
3375  0.573855  0.367268  0.511637  0.573782  0.539454  0.551808  0.451396   
3376  0.789643  0.573959  0.472380  0.700525  0.423384  0.396176  0.469704   
3377  0.624761  0.479428  0.594044  0.558092  0.488441  0.574005  0.674658   
3378  0.572060  0.438443  0.511442  0.518009  0.454172  0.499817  0.497614   
3379  0.536977  0.606614  0.693947  0.728551  0.475182  0.544432  0.712511   

            7         8         9      

In [34]:
#Balancear datos


In [35]:


X = np.array(df_normalizado.drop([17],1))
y = np.array(df_normalizado[17])


<ipython-input-35-69ced6404877>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = np.array(df_normalizado.drop([17],1))


In [36]:
# Verificar desequilibrio en las clases
desequilibrio_clases = df[17].value_counts()
print("Distribución de clases:")
print(desequilibrio_clases)
# 1= enfermo
# 2= sano

Distribución de clases:
1    1724
0    1656
Name: 17, dtype: int64


In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento, validación y prueba
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Definir la forma del tensor de entrada
input_shape = (X_train.shape[1], 1)  # (número de características, número de canales)

# Reshape de X para que tenga la forma adecuada para la entrada de la CNN
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Definir el modelo de CNN
model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
    MaxPooling1D(pool_size=2),
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Capa de salida binaria
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
# Calcular recall y F1-score
# Predecir las probabilidades de pertenencia a la clase positiva
y_pred_prob = model.predict(X_test)
# Convertir las probabilidades en clases usando un umbral de decisión (0.5 en este caso)
y_pred = (y_pred_prob > 0.5).astype(int)



recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Pérdida en el conjunto de prueba: {loss}')
print(f'Precisión en el conjunto de prueba: {accuracy}')
print("recall_score :",recall)
print("f1_score :",f1)

Epoch 1/10
85/85 [==============================] - 2s 9ms/step - loss: 0.6908 - accuracy: 0.5370 - val_loss: 0.6800 - val_accuracy: 0.6391
Epoch 2/10
85/85 [==============================] - 1s 6ms/step - loss: 0.6736 - accuracy: 0.5754 - val_loss: 0.6412 - val_accuracy: 0.6509
Epoch 3/10
85/85 [==============================] - 0s 4ms/step - loss: 0.6404 - accuracy: 0.6487 - val_loss: 0.6145 - val_accuracy: 0.6657
Epoch 4/10
85/85 [==============================] - 0s 3ms/step - loss: 0.6239 - accuracy: 0.6572 - val_loss: 0.6075 - val_accuracy: 0.6627
Epoch 5/10
85/85 [==============================] - 0s 3ms/step - loss: 0.6225 - accuracy: 0.6520 - val_loss: 0.6030 - val_accuracy: 0.6716
Epoch 6/10
85/85 [==============================] - 0s 3ms/step - loss: 0.6082 - accuracy: 0.6727 - val_loss: 0.5928 - val_accuracy: 0.6923
Epoch 7/10
85/85 [==============================] - 0s 3ms/step - loss: 0.6082 - accuracy: 0.6709 - val_loss: 0.6213 - val_accuracy: 0.6183
Epoch 8/10
85/85 [==